In [ ]:
import caffe
import lmdb
import numpy as np
import cv2
import io
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rotate_image(im, angle):
    rows,cols = im.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    return cv2.warpAffine(im,M,(cols,rows))

def shuffle_samples_lmdb(path_lmdb, path_dst, keys):
    """
    Copy select samples from an lmdb into another.
    Can be used for sampling from an lmdb into another and generating a random shuffle
    of lmdb content.
    
    Parameters:
    path_lmdb -- source lmdb
    path_dst -- destination lmdb
    keys -- list of keys or indices to sample from source lmdb
    """
    MAP_SZ = int(1e12)
    NUM_IDX_DIGITS = 10
    IDX_FMT = '{:0>%d' % NUM_IDX_DIGITS + 'd}'
    db = lmdb.open(path_dst, map_size=MAP_SZ)
    key_dst = 0
    with db.begin(write=True) as txn_dst:
        with lmdb.open(path_lmdb, readonly=True).begin() as txn_src: 
            for key_src in keys:
                ########################
                if not isinstance(key_src, basestring):
                    key_src = IDX_FMT.format(key_src)
                txn_dst.put(IDX_FMT.format(key_dst), txn_src.get(key_src))
                key_dst += 1
   
                
                ########################
                
            #if not isinstance(key_src, basestring):
                #    key_src = IDX_FMT.format(key_src)
                #    
                #datum_from = caffe.proto.caffe_pb2.Datum()#

            #    datum_from.ParseFromString(txn_src.get(key_src))

             #   datum_to = caffe.proto.caffe_pb2.Datum()
               # datum_to.label = datum_from.label
              #  datum_to.data = datum_from.data
                
                #txn_dst.put(IDX_FMT.format(key_dst),  datum_to.SerializeToString())
                #key_dst += 1
                # if key_dst ==5:
                # break
    db.close()
    
    

    
def create_shuffled_ind(lmdb_dir, txt_save_dir):
    NUM_IDX_DIGITS = 10
    IDX_FMT = '{:0>%d' % NUM_IDX_DIGITS + 'd}'
    db = lmdb.open(lmdb_dir)
    N_im = db.stat()['entries']
    ind = range(N_im)
    print('Indices created', N_im)
    np.random.shuffle(ind)
    keys_shuffled = [IDX_FMT.format(i) for i in ind]

    import json
    if not os.path.exists(txt_save_dir):
        os.makedirs(txt_save_dir)
    
    f = open(txt_save_dir + '/indices_list.txt', 'w')
    json.dump(keys_shuffled, f)
    f.close()
    return ind

def load_N_MNIST_images(N,path):
    #path = '/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_train_lmdb'
    lmdb_env = lmdb.open(path)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()
    datum_db = caffe.proto.caffe_pb2.Datum()
    I = []
    Label = []
    im_count = 0
    for key, value in lmdb_cursor:
            datum_db.ParseFromString(value)
            label = datum_db.label
            data = caffe.io.datum_to_array(datum_db)
            im = data.astype(np.uint8)
            im = np.transpose(im, (2, 1, 0)) # original (dim, col, row)
            I.append(im.squeeze())
            Label.append(datum_db.label)
            im_count = im_count + 1
            
            if im_count >= N:
                break
    return I, Label



In [ ]:
IMG = load_N_MNIST_images(10,'/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_train_lmdb')
#for i in range(IMG.shape[2]):
#    plt.imshow(IMG[:,:,i].T)
#    plt.show()

#for key, value in lmdb_cursor:
#            datum_db.ParseFromString(value)
#            label = datum_db.label
#            data = caffe.io.datum_to_array(datum_db)
#            im = data.astype(np.uint8)
#            im = np.transpose(im, (2, 1, 0)) # original (dim, col, row)
#            im_count = im_count + 1
#            if im_count > N_im_total:
#                break
#            # Loop over images in database
print(IMG[1].shape)

In [ ]:
set_strings = ['_TRAIN_', '_TEST_ON_TRAIN_', '_TEST_']
dir_strings = ['_train_', '_train_', '_test_']
NUM_IDX_DIGITS = 10
IDX_FMT = '{:0>%d' % NUM_IDX_DIGITS + 'd}'

N_im = 1000  #Size of images used from MNIST
batch_size = 1000
home_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder'

for set_st, dir_str in zip(set_strings, dir_strings):
    set_str = set_st + str(N_im) + '_'

    #Create angles
    angles = range(-90,90+1,15)

    #Load MNIST Database
    IMG, LABEL = load_N_MNIST_images(N_im,'/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist' + dir_str + 'lmdb')
   # N_im = db.stat()['entries']


    #N_im = lmdb_env.stat()['entries']
    N = np.dot(np.shape(angles),N_im)
    N = N[0]
    # Let's pretend this is interesting data
    #X = np.zeros((N, 1, 28, 28), dtype=np.uint8)
    #y = np.zeros(N, dtype=np.int64)


    map_size = N*28*28*100

    # Transformed MNIST Database
    #env_rot = lmdb.open(home_dir + '/MNIST' + set_str + 'rot_lmdb/', map_size=map_size)
    #env_rot_ang = lmdb.open(home_dir + '/MNIST' + set_str + 'rot_ang_lmdb/',map_size=int(1e12))
    #datum = caffe.proto.caffe_pb2.Datum()


    rot_angs = np.zeros(N)
    im_count = 0
    #Looping over batches
    for idx in range(int(np.ceil(N/batch_size))):
        count = 0
        env_rot = lmdb.open(home_dir + 'MNIST' + set_str + 'rot_lmdb/', map_size=map_size)
        env_rot_ang = lmdb.open(home_dir + 'MNIST' + set_str + 'rot_ang_lmdb/',map_size=int(1e12))
        env_unrot_ang = lmdb.open(home_dir + 'MNIST' + set_str + 'unrot_lmdb/',map_size=map_size)

        with env_rot.begin(write=True) as txn:    
            with env_unrot_ang.begin(write=True) as txn_unrot_ang:
                with env_rot_ang.begin(write=True) as txn_rot_ang:
                    # Looping over images from MNIST
                    for  in_, lab_ in zip(IMG[(batch_size*idx):(batch_size*(idx+1))], LABEL[(batch_size*idx):(batch_size*(idx+1))]):
                        im = in_
                        label = lab_
                        for angle in angles:
                            #Prepare data
                            X = rotate_image(im.squeeze(), angle)
                            X = np.reshape(X,[1,28,28])
                            
                            datum = caffe.io.array_to_datum(X.astype(float), label)
                           #  if in_idx ==1:
                           #     print(X.astype(float).shape)
                            #datum.channels = X.shape[0]
                            #datum.height = X.shape[1]
                            #datum.width = X.shape[2]
                            #datum.data = X.tostring()  # or .tostring() if numpy < 1.9

                            #datum.label = label
                            rot_angs[count] = str(angle) # "{0}".format(angle)
                            str_id = IDX_FMT.format(batch_size*len(angles)idx + count)
                            #if count%100==0:
                            #    print(str_id)
                            #print(datum)

                            # The encode is only essential in Python 3
                            txn.put(str_id, datum.SerializeToString())
                            txn_rot_ang.put(str_id, str(angle) )
                            
                            #Storing unroted images
                            datum_unrot = caffe.io.array_to_datum(np.reshape(im.squeeze(),[1,28,28]).astype(float), label)

                            txn_unrot_ang.put(str_id, datum_unrot.SerializeToString()) 
                            count = count + 1
                            
                env_rot_ang.close()
            env_unrot_ang.close()
        env_rot.close()
        
    src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb'
    dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled'

    ind = create_shuffled_ind(src_dir, dst_dir)
    
    shuffle_samples_lmdb(src_dir, dst_dir, ind)

    src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb'
    dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb/shuffled'
    shuffle_samples_lmdb(src_dir, dst_dir, ind)

    src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'unrot_lmdb'
    dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'unrot_lmdb/shuffled'
    shuffle_samples_lmdb(src_dir, dst_dir, ind)



In [ ]:
lmdb_txn = env_rot.begin()
lmdb_cursor = lmdb_txn.cursor()
datum = caffe.proto.caffe_pb2.Datum()
i=0
for key, value in lmdb_cursor:
    datum.ParseFromString(value)
    print(key)
    label = datum.label
    data = caffe.io.datum_to_array(datum)
    im = data.astype(np.uint8)
    im = np.transpose(im, (2, 1, 0)) # original (dim, col, row)
    print(im.shape)
    print "label ", label

    print(im.shape)
    i= i+1
    if i ==3:
        break

In [ ]:
lmdb_env = lmdb.open('/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_train_lmdb')
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()
datum = caffe.proto.caffe_pb2.Datum()

#lmdb_shuf = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST_rot_lmdb/shuffled')
#lmdb_txn_shuf = lmdb_shuf.begin()
#lmdb_cursor_shuf = lmdb_txn_shuf.cursor()


lmdb_rot = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str +'rot_lmdb')
lmdb_txn_rot = lmdb_rot.begin()
lmdb_cursor_rot = lmdb_txn_rot.cursor()


lmdb_rot_ang = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled/')
lmdb_txn_rot_ang = lmdb_rot_ang.begin()
lmdb_cursor_rot_ang = lmdb_txn_rot_ang.cursor()

i=0
for key, value in lmdb_cursor:
    datum.ParseFromString(value)
    #print(datum.channels)
    label = datum.label
    data = caffe.io.datum_to_array(datum)
    im1 = data.astype(np.uint8)
    plt.imshow(im1.squeeze())
    plt.title(label)
    plt.show()
    #print(im.sha
    #im1 = np.transpose(im, (2, 1, 0)) # original (dim, col, row)
    #print("label ", label)
    #plt.imshow(im1.squeeze())
    #plt.title(label)
    #plt.show()
    #print(im.shape)
    i= i+1
    if i ==5:
        i=0
        break
         
print('rotated')
        
for  key, value in lmdb_cursor_rot_ang: 
    #print(key, value)
    datum.ParseFromString(value)
    #print(datum.channels)
    label = datum.label
    data = caffe.io.datum_to_array(datum)
    im = data.astype(np.uint8)
    #im = np.transpose(im, (2, 1, 0)) # original (dim, col, row)
    #print("label ", label)
    # print(key,value)
    # print(im.shape)
    plt.imshow(im.T.squeeze())
    plt.title(str(label) + '_' + ang_rot)
    #print(key_rot, ang_rot)
    #print(key,value)
    
    plt.show()
    #print(im.shape)
    i= i+1
    if i ==10:
        break

In [ ]:
print(set_str)
#shuffled_ind = open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST_rot_lmdb/mylmdb/keys_shuffled.txt', 'r')
src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb'
dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled'
    
ind = create_shuffled_ind(src_dir, dst_dir)

shuffle_samples_lmdb(src_dir, dst_dir, ind)

src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb'
dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb/shuffled'
shuffle_samples_lmdb(src_dir, dst_dir, ind)

In [ ]:
lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str 'rot_lmdb')
np.shape(ind)

In [ ]:
#N_im_total = 1000
N_im = 1000
set_str = '_TRAIN_' + str(N_im) + '_'
lmdb_env = lmdb.open('/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_test_lmdb')
lmdb_shuf = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled/')
lmdb_rot = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str +'rot_lmdb/')
lmdb_rot_ang = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb/')
lmdb_rot_ang_shuf = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb/shuffled/')
print(lmdb_env.stat())
print(lmdb_shuf.stat())
print(lmdb_rot.stat())
print(lmdb_rot_ang.stat())
print(lmdb_rot_ang_shuf.stat())

In [ ]:
print('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled')

In [ ]:
angles = range(90,180+1,10)
print(angles,len(angles))

In [ ]:
plt.imshow(im1.squeeze())
plt.show()
plt.imshow(rotate_image(im1.squeeze(),90))
plt.show()

In [ ]:
len(np.linspace(-90,90,10))


In [ ]:
lmdb_rot_ang = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST_rot_ang_lmdb')
lmdb_txn_rot_ang = lmdb_rot_ang.begin()
lmdb_cursor_rot_ang = lmdb_txn_rot_ang.cursor()
i=0
for a,b in lmdb_cursor_rot_ang:
    print(a,b)
    i=i+1
    if i==10:
        break

In [ ]:
angles = range(-90,90+1,15)
print(angles,len(angles))

In [ ]:
range(-90,90+1,30)


In [ ]:

if im_count < N_im_total:
    print('A')

In [ ]:
range(int(np.ceil(100/10)))

In [ ]:
IMG, LABEL = load_N_MNIST_images(N_im_total,'/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_train_lmdb')


In [ ]:
set_str = '_TRAIN_' + str(10000) + '_'

src_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb'
dst_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled'
ind = create_shuffled_ind(src_dir, dst_dir)

shuffle_samples_lmdb(src_dir, dst_dir, ind)

In [ ]:
'/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_ang_lmdb/shuffled/'

In [ ]:
i = 0
lmdb_rot_ang = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/')
lmdb_txn_rot_ang = lmdb_rot_ang.begin()
lmdb_cursor_rot_ang = lmdb_txn_rot_ang.cursor()


for key, value in lmdb_cursor_rot_ang:
    #print(key,value)
    with lmdb_rot_ang.begin() as txn_bla: 
        raw_datum = txn_bla.get(key)
        datum = caffe.proto.caffe_pb2.Datum()
        datum.ParseFromString(raw_datum)
        #print(datum.data)
        flat_x = np.fromstring(datum.data, dtype=np.uint8)
        print(flat_x.shape)
        plt.imshow(flat_x.squeeze())
        plt.show()
        print(datum.label)
        i=i+1
        if i == 20:
            break


In [ ]:

set_str = '_TRAIN_1000_'
env = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder/MNIST' + set_str + 'rot_lmdb/shuffled/', readonly=True)
with env.begin() as txn:
    raw_datum = txn.get(b'0000000003')

datum = caffe.proto.caffe_pb2.Datum()
datum.ParseFromString(raw_datum)

flat_x = np.fromstring(datum.data, dtype=np.uint8)
x = flat_x
y = datum.label
#data2 = caffe.io.datum_to_array(raw_datum)
y
print(y, raw_datum)

In [ ]:
data = caffe.io.datum_to_array(datum.data)
y

In [ ]:
IMG, LABEL = load_N_MNIST_images(1000,'/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/mnist_test_lmdb')

In [ ]:
print(len(range(-90,90+1,15))*5000)


In [ ]:
17000/5000.


In [ ]:
A = lmdb.open('/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/MNIST_lmdb/MNIST_TRAIN_5000_unrot_lmdb')
A.stat()

In [ ]:
5000